# Getting started with InferLO

In this Notebook we will review basic concepts, show how to define graphical models in InferLO, how to solve different tasks (inference, maximum likelihood and sampling) for that model. Also we will review different kinds of models (generic, pairwise and Normal Factor Graph), and show how you can use different algorithms to solve the same problem.

In [1]:
# Don't run this cell.
%load_ext autoreload
%autoreload 2
import sys 
sys.path.append('../..')

In [3]:
import inferlo
import numpy as np
from matplotlib import pyplot as plt

## Introduction

Essentially graphical model is a way to represent a probabilistic distribution over variables. This distribution is usually defined by non-normalized probability dnsity function, represented as a product of factors, each factor being a function of small number of variables. Then we can define a graph describing structure of those factors (there are multiple ways to do so).

In InferLO you define a model in terms of two basic building blocks: `Variable`s and `Factor`s. First we review how you can do it by defining them explicilty.

## Defining a model

Let's start by defining a generic graphical model, which is explicitly defined by (non-normalized) probability density function. Let's say we have 4 variables $x_0, x_1, x_2, x_3$ which can take any value, and we want to represent probability distribution


$$p(x) \sim e^{x_1 \cdot x_2} \cdot |x_2+x_3| \cdot \cos(x3-x4)^2.$$

For this we will use `GenericGraphModel`. First, we should initialize the model with number of variables. Then, we will add factors add by one. 

In this case we will use `FunctionFactor`, which defines factor explicilty by indices of variables on which it depends and a function. To define such a factor we need three things: a model, indices of variables and a function. Function should take a list of values (its length must match number of variables this factor depends on) and evaluate value of this factor for given values of variables.

In [ ]:
model = inferlo.GenericGraphModel(4)
model.add_factor(inferlo.FunctionFactor(model, [0, 1], lambda x: np.exp(x[0] * x[1])))
model.add_factor(inferlo.FunctionFactor(model, [1, 2], lambda x: np.abs(x[0] + x[1])))
model.add_factor(inferlo.FunctionFactor(model, [2, 3], lambda x: np.cos(x[0] - x[1])**2))

Now, we can evaluate value of the whole product at any point:

In [ ]:
model.evaluate([1,2,3,4])

We can define the same model in more succint way, using symbolic expressions. Note that `*=` is just shortcut for `add_factor` and expression on the right creates a `FunctionFactor`.

In [ ]:
model = inferlo.GenericGraphModel(4)
x0, x1, x2, x3 = model.get_symbolic_variables()
model *= np.exp(x0*x1)
model *= abs(x1+x2)
model *= np.cos(x2-x3)**2
model.evaluate([1,2,3,4])

## Variable domains

In example above variables could take any real value. In practice it's useful to have variables with limited domain. You can specify domain for every individual variable in the model. Note that `model[i]` accesses `i`-th variable:

In [ ]:
model = inferlo.GenericGraphModel(4)
model[0].domain = inferlo.RealDomain()
model[1].domain = inferlo.DiscreteDomain([10, 100, 1000])
model[2].domain = inferlo.DiscreteDomain.binary()
model[3].domain = inferlo.DiscreteDomain.range(5)

print([var.domain for var in model.get_variables()])

In this example $x_0 \in \mathbb{R}, x_1 \in \{10, 100, 1000\}, x_2 \in \{0, 1\} , x_3 \in \{0, 1,2,3,4,5\}$. 

By default domains of all variables are real. If you want to change domains of some variables, please do so first thing after you created the model. 

If you want all variables to have the same (default) domain, you can specify this domain when creating a model.
In example below all variables are binary, except last one, which can take values 0,1 or 2.

In [ ]:
model = inferlo.GenericGraphModel(5, domain=inferlo.DiscreteDomain.binary())
model[4].domain = inferlo.DiscreteDomain.range(3)
print([var.domain for var in model.get_variables()])

## Factors

In example above we used `FunctionFactor`, which is defined explicilty by a function, and it can be any function. There are other ways to represent factors.

If all variables, on which the fcator depends, are discrete, number of possible combinations of variable values is finite, so you can tabulate the function and represnt it by this table (which will be a multidimensional array). You can do this in InferLO by using a `DiscreteFactor`.

In [ ]:
model = inferlo.GenericGraphModel(2, domain=inferlo.DiscreteDomain.binary())
factor = inferlo.DiscreteFactor(model, [0, 1], [[0, 1], [2, 3]])
factor.value([1, 0])

Above we described a function $f(x_0, x_1)$ such that $f(0,0)=0, f(0,1)=1, f(1,0)=2, f(1,1)=3$.

You can define a function explicilty (using `FunctionFactor`) and then convert it do `DiscreteFactor`, obtaining explict table of function values:

In [ ]:
model = inferlo.GenericGraphModel(3, domain=inferlo.DiscreteDomain.binary())
model[2].domain = inferlo.DiscreteDomain([2, 3, 4])
x0, x1, x2 = model.get_symbolic_variables()
function_factor = 100 * x0 + 10 * x1 + x2
discrete_factor = inferlo.DiscreteFactor.from_factor(function_factor)
discrete_factor.values

## Pairwise models

Generic model can express any distribution, but it's too abstract. InferLO defines classes for more concrete models, which may be subjects to some additional constraints, but allow for efficient algorithms to solve certain problems.

First such example is Pairwise Finite model --- model where every factor depends on at most 2 variables, and where variables come from the same discrete domain. Famous partial cases of this kind of model are Potts model and Ising model.



## Normal Factor Graph models.

If every variable appears in exactly two factors, we can build a Normal Factor Graph - a graph where edges correspond to variables, and vertices correspond to factors. We we will call such models Normal Factor Graph (NFG) models. Also they are known ad Edge-Variable or Forney-style models.

Not any model can be represented in such a way, but every model can be converted to NFG model (conversion may add new variables and factors).

## Conversion from factor graph to Forney-style edge-variable graph and back

In [ ]:
model1 = inferlo.GenericGraphModel(num_variables = 5)
x1, x2, x3, x4, x5 = inferlo.FunctionFactor.prepare_variables(model1)
model1 *= np.exp(x1*x2)
model1 *= np.exp(x1*x3*x4)
model1 *= np.exp(x2*x3*x4)
model1 *= np.exp(x2*x3*x5)

for i in range(len(model1.factors)):
    model1.factors[i].name = 'f%d' % i
    
fig = plt.figure(figsize=(30, 10))
model1.draw_factor_graph(fig.add_subplot(1, 3, 1)) 


model2 = inferlo.NormalFactorGraphModel.from_model(model1) 
model2.draw_edge_variable_graph(fig.add_subplot(1, 3, 2))
model2.draw_factor_graph(fig.add_subplot(1, 3, 3))

ax = fig.get_axes()
ax[0].set_title('Initial factor graph')
ax[1].set_title('Equivalent Forney-style graph')
ax[2].set_title('Factor graph for converted model')
plt.show()

## Conversion from pairwise to factor-graph and Forney-style

In [ ]:
field = np.zeros((4, 2))
edges = [[0, 1], [0, 2], [0, 3]]
j1 = np.array([[0, 0], [0, 1]])
interactions = [j1, j1, j1]
model1 = inferlo.PairWiseFiniteModel.create(field, edges, interactions)

fig = plt.figure(figsize=(30, 10))
model1.draw_pairwise_graph(fig.add_subplot(1, 3, 1))

model1.draw_factor_graph(fig.add_subplot(1, 3, 2))

model2 = inferlo.NormalFactorGraphModel.from_model(model1) 
model2.draw_edge_variable_graph(fig.add_subplot(1, 3, 3))

ax = fig.get_axes()
ax[0].set_title('Pairwise graph')
ax[1].set_title('Factor graph')
ax[2].set_title('Edge-varable graph')
plt.show()

## Inference / optimization.

In [ ]:
model = inferlo.PairWiseFiniteModel(5, 3)
x = inferlo.FunctionFactor.prepare_variables(model)
model *= np.exp(3*x[0]*x[1])
model *= np.exp(4*x[1]*x[2])
model *= np.exp(-2*x[2]*x[3])
model *= np.exp(-6*x[2]*x[4])
model *= np.exp(x[3]*x[0])
model.draw_pairwise_graph(plt)

# Also can explictly set interactions as 3x3 matrices.

model.infer(algorithm='message_passing')
model.max_likelihood()